In [ ]:
import pandas as pd
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/이브와 모델 시도/전치리완료df.csv')
# df.reset_index(level=0, inplace=True)  # 인덱스 값을 컬럼으로 변환
# df.rename(columns={'index': 'idx'}, inplace=True)  # 컬럼명 변경
df

In [ ]:
!pip install -q pytorch-lightning==1.9.4
!pip install -q transformers==4.20.0
!pip install -q wandb
!pip install sentencepiece
# # !pip install py-hanspell


In [ ]:
import transformers
from transformers import AutoModelForSequenceClassification,AutoTokenizer, AutoConfig,AdamW
# transformers.__version__

import os
os.cpu_count()

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

from imblearn.over_sampling import RandomOverSampler

import torch
from torch.utils.data import Dataset, SubsetRandomSampler, DataLoader
from sklearn.metrics import accuracy_score
import pytorch_lightning as pl
from pytorch_lightning import LightningDataModule
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor
from pytorch_lightning.loggers import WandbLogger

import wandb

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('wordnet2022')

In [ ]:
my_config = {
    'project_name': 'voice_pp',
    'model_name': 'microsoft/deberta-v3-large',
    'num_workers': 0,  # adjust as needed
    'num_labels': 2,
    'max_epochs': 20,
    'patience' : 2,
    'max_length': 512,
    'batch_size': 3,
    'learning_rate':6e-6,
    'accumulate_grad_batches': 4,
    'warmup_steps': None,  # to be set later
    'wandb_api_key': '본인의 키 넣기',
    'data_dir': '/content/drive/MyDrive/이브와 모델 시도/data_dir',
    'result_dir': '/content/drive/MyDrive/이브와 모델 시도/result_dir'
}

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
train_df = train_df.reset_index(drop=True)
train_df = train_df.reset_index()
train_df = train_df.rename(columns = {"index":"idx"})
test_df= test_df.reset_index(drop=True)
test_df = test_df.reset_index()
test_df = test_df.rename(columns = {"index":"idx"})
# train_df[""]

# test_dff

In [ ]:
train_df = train_df.drop_duplicates()
test_df = test_df.drop_duplicates()

In [ ]:
class NewsDataset(Dataset):
    def __init__(self, df, tokenizer, max_length, train_or_test='train'):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.train_or_test = train_or_test


    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.loc[idx, 'comment']
        inputs = self.tokenizer(text,
                                add_special_tokens=True,
                                max_length=self.max_length,
                                padding='max_length',
                                truncation=True,
                                return_tensors='pt')

        if self.train_or_test == 'train':
            label = self.df.loc[idx, 'label']
            return {
                'input_ids': inputs['input_ids'].squeeze(),
                'attention_mask': inputs['attention_mask'].squeeze(),
                'labels': torch.tensor(label, dtype=torch.long)
            }
        else:
            return {
                'input_ids': inputs['input_ids'].squeeze(),
                'attention_mask': inputs['attention_mask'].squeeze(),
                'idx': self.df.loc[idx, 'idx']
            }


In [ ]:
class NewsDataModule(LightningDataModule):
    def __init__(self, config, tokenizer):
        super().__init__()
        self.config = config
        self.tokenizer = tokenizer
        self.batch_size = config['batch_size']
        self.num_workers = config['num_workers']

    def prepare_data(self):
        # Load train and test data
        self.train_df = train_df
        self.test_df = test_df

    def setup(self, stage=None):
        # Split train data into train and validation
        train_data, val_data = train_test_split(self.train_df, test_size=0.2, random_state=43, stratify=self.train_df['label'])

        #Apply RandomOverSampler
        ros = RandomOverSampler(random_state=43)
        X_resampled, y_resampled = ros.fit_resample(train_data[['idx', 'comment']], train_data['label'])

        # Concatenate X_resampled and y_resampled
        resampled_data = pd.concat([X_resampled, y_resampled], axis=1)

        # Reset the index
        resampled_data.reset_index(drop=True, inplace=True)
        val_data.reset_index(drop=True, inplace=True)
        self.test_df.reset_index(drop=True, inplace=True)

        # Calculate the number of samples to load per epoch
        self.num_samples_per_epoch = len(resampled_data) // 5
        self.train_indices = list(range(len(resampled_data)))

        # Create datasets
        self.train_dataset = NewsDataset(resampled_data, self.tokenizer, self.config['max_length'], 'train')
        self.val_dataset = NewsDataset(val_data, self.tokenizer, self.config['max_length'], 'train')
        self.test_dataset = NewsDataset(self.test_df, self.tokenizer, self.config['max_length'], 'test')

    def train_dataloader(self):
        sampler = SubsetRandomSampler(self.train_indices[:self.num_samples_per_epoch])
        return DataLoader(self.train_dataset,
                          batch_size=self.batch_size,
                          sampler=sampler,
                          num_workers = my_config['num_workers']
                         )

    def val_dataloader(self):
        return DataLoader(self.val_dataset,
                          batch_size=self.batch_size,
                          num_workers = my_config['num_workers']
                         )

    def test_dataloader(self):
        return DataLoader(self.test_dataset,
                          batch_size=self.batch_size,
                          num_workers = my_config['num_workers']
                         )

In [ ]:
class NewsClassifier(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.model = AutoModelForSequenceClassification.from_pretrained(config['model_name'], num_labels=config['num_labels'])

    def forward(self, input_ids, attention_mask, labels=None):
        return self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

    def training_step(self, batch, batch_idx):
        outputs = self(batch['input_ids'], batch['attention_mask'], batch['labels'])
        loss = outputs.loss
        self.log('train_loss',loss)
        return {'loss': loss}


    def validation_step(self, batch, batch_idx):
        outputs = self(batch['input_ids'], batch['attention_mask'], batch['labels'])
        loss = outputs.loss
        logits = outputs.logits.argmax(dim=1)
        labels = batch['labels'].cpu()

        f1 = f1_score(labels, logits.cpu(), average='macro')
        acc = accuracy_score(labels, logits.cpu())

        self.log('val_loss', loss)
        self.log('val_f1_score', f1)
        self.log('val_acc', acc)  # Log accuracy as 'val_acc'

        return {'val_loss': loss, 'val_f1_score': f1, 'val_acc': acc}

    def test_step(self, batch, batch_idx):
        outputs = self(batch['input_ids'], batch['attention_mask'])
        logits = outputs.logits.argmax(dim=1)
        labels = batch['labels'].cpu
        loss = outputs.loss
        f1 = f1_score(labels, logits.cpu(), average='macro')
        acc = accuracy_score(labels, logits.cpu())

        self.log('test_loss', loss)
        self.log('test_f1_score', f1)
        self.log('test_acc', acc)  # Log accuracy as 'val_acc'
        return {'test_loss': loss, 'test_f1_score': f1, 'test_acc': acc}

    def test_epoch_end(self, outputs):
        ids = []
        labels = []
        for out in outputs:
            ids.extend(out['id'])
            labels.extend(out['label'].cpu().tolist())  # Move the label tensor to CPU
        # submission = pd.DataFrame({'id': ids, 'label': labels})
        # submission.to_csv(os.path.join(self.config['result_dir'], 'submission_debertav3.csv'), index=False)

        return submission

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.config['learning_rate'])
        return optimizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(my_config['model_name'])

# Create dataset
dm = NewsDataModule(my_config, tokenizer)
dm.prepare_data()
dm.setup()

In [ ]:
!wandb login

In [ ]:
wandb.login(key=my_config['wandb_api_key'])
wandb_logger = WandbLogger(name = 'colab_full_debertav3',
                           project=my_config['project_name']
                          )


early_stopping = EarlyStopping(monitor='val_f1_score', mode='max', patience=my_config['patience'])
checkpoint_callback_f1 = ModelCheckpoint(monitor='val_f1_score', mode='max', save_top_k=1, dirpath=my_config['result_dir'], filename='best_model')
lr_monitor = LearningRateMonitor(logging_interval='step')

In [ ]:
trainer = pl.Trainer(
    logger=wandb_logger,
    gpus=1,
    max_epochs=my_config['max_epochs'],
    callbacks=[early_stopping, checkpoint_callback_f1, lr_monitor],
    precision=16,
    accumulate_grad_batches=my_config['accumulate_grad_batches'],
)


# Train the model
news_classifier = NewsClassifier(my_config)
# trainer.fit(news_classifier, dm)

In [ ]:
# Test the model and save the submission results
submission_results = trainer.test(news_classifier, datamodule=dm, ckpt_path='/content/drive/MyDrive/이브와 모델 시도/result_dir/best_model.ckpt')

print(submission_results)


In [ ]:
trainer.test(news_classifier,
             datamodule=dm,
             ckpt_path='/content/drive/MyDrive/이브와 모델 시도/result_dir/best_model.ckpt'
             )